In [ ]:
!pip install urllib3==1.25.10
!pip install pytorch-pretrained-bert

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1KWiNovt1kir-C0tlRKIg0KiP6VKNqzuD' -O "data.csv"
#https://drive.google.com/file/d/1KWiNovt1kir-C0tlRKIg0KiP6VKNqzuD/view?usp=sharing

--2020-11-17 18:18:19--  https://docs.google.com/uc?export=download&id=1KWiNovt1kir-C0tlRKIg0KiP6VKNqzuD
Resolving docs.google.com (docs.google.com)... 142.250.97.139, 142.250.97.113, 142.250.97.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.97.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hi04ok6k231v65eh9n7vro2b4t6b5ccv/1605637050000/04261418863817579276/*/1KWiNovt1kir-C0tlRKIg0KiP6VKNqzuD?e=download [following]
--2020-11-17 18:18:19--  https://doc-10-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hi04ok6k231v65eh9n7vro2b4t6b5ccv/1605637050000/04261418863817579276/*/1KWiNovt1kir-C0tlRKIg0KiP6VKNqzuD?e=download
Resolving doc-10-94-docs.googleusercontent.com (doc-10-94-docs.googleusercontent.com)... 74.125.31.132, 2607:f8b0:400c:c02::84
Connecting to doc-10-94-docs.googleusercontent.com (doc-10-94-do

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,web-scraper-order,text,section,topic,urls,userMentions,isReply,emojis,Expression of emotions,Use of humor,Self-disclosure,Referring explicitly to others' messages,Asking questions,"Complimenting, expressing appreciation or criticism",Expressing agreement or disagreement,"Phatics, salutations"
0,1604198662-928,Hi did you fix this bug?\n,bugreports,Subject 11057234,NaN,NaN,True,NaN,0,0,0,0,1,0,0,1
1,1604198796-1345,I have never been quite certain that the X ax...,bugreports,RFE: An easy way to read off feature frequencies,NaN,NaN,NaN,NaN,0,0,1,0,1,0,0,0
2,1604198776-1275,\nBecause I can understand it is of immense in...,bugreports,Statistics not updating on Gravity Spy home page,NaN,NaN,True,😦,1,0,1,0,0,0,0,0
3,1604198884-1593,"And working again.\n(A Schrödinger tool, as we...",bugreports,Gravity Spy Tools down,NaN,NaN,NaN,😃,1,1,1,0,0,0,0,0
4,1604197902-62,I did occasionally. Reloading the site helps t...,bugreports,Already Seen keeps coming up,/users/eperozzi;/users/sbc538,@eperozzi;@sbc538,NaN,NaN,0,0,0,1,0,0,0,0


In [ ]:
Category = "Complimenting, expressing appreciation or criticism"
Data = 'SP'
TASK_NAME = Data+Category

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
df = shuffle(df, random_state=26)
train_df, test_df = train_test_split(df, test_size=0.2,stratify = df[Category])

In [ ]:
train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df[Category],
    'alpha':['a']*train_df.shape[0],
    'text': train_df['text'].replace(r'\n', ' ', regex=True)
})
train_df_bert.head()

,id,label,alpha,text
25,0,0,a,Wow! A perfect #mirrorchirp. I want to know: i...
370,1,0,a,20885054.0
83,2,0,a,repeating low freq bursts
209,3,0,a,Possibly a very unusual light modulation
115,4,0,a,#scratchy #noiseband


In [ ]:
dev_df_bert = pd.DataFrame({
    'id':range(len(test_df)),
    'label':test_df[Category],
    'alpha':['a']*test_df.shape[0],
    'text': test_df['text'].replace(r'\n', ' ', regex=True)
})
dev_df_bert.head()

,id,label,alpha,text
297,0,0,a,#suspect lfb?
381,1,0,a,0.2 to 0.4 #whiplash glitch seen many times n...
151,2,0,a,Are the subjects numbered sequentially in time...
223,3,0,a,#lowfrequencyhelix ?
196,4,0,a,


In [ ]:
w = list(train_df_bert['label'].value_counts())
wb1 = []
for i in range(len(w)):
    wb1.append(1/(w[i]/(len(w)*sum(w))))

In [ ]:
import os
#os.mkdir('/data')

In [ ]:
train_df_bert.to_csv(f'/data/train.tsv', sep='\t', index=False, header=False)
dev_df_bert.to_csv(f'/data/dev.tsv', sep='\t', index=False, header=False)

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wwo5wmuVhDQBSg_11FGOrF1Ve2VPCXcH' -O "tools.py"
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rJtgFZLPH_HUCypuU2uXaCfG-Zx2x-wO' -O "convert_examples_to_features.py"

--2020-11-17 18:20:59--  https://docs.google.com/uc?export=download&id=1wwo5wmuVhDQBSg_11FGOrF1Ve2VPCXcH
Resolving docs.google.com (docs.google.com)... 64.233.170.101, 64.233.170.102, 64.233.170.113, ...
Connecting to docs.google.com (docs.google.com)|64.233.170.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/04b4fkoig8k09or01eaj6j3c23udlfjl/1605637200000/07086883988310421371/*/1wwo5wmuVhDQBSg_11FGOrF1Ve2VPCXcH?e=download [following]
--2020-11-17 18:20:59--  https://doc-14-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/04b4fkoig8k09or01eaj6j3c23udlfjl/1605637200000/07086883988310421371/*/1wwo5wmuVhDQBSg_11FGOrF1Ve2VPCXcH?e=download
Resolving doc-14-64-docs.googleusercontent.com (doc-14-64-docs.googleusercontent.com)... 74.125.31.132, 2607:f8b0:400c:c02::84
Connecting to doc-14-64-docs.googleusercontent.com (doc-14-64-do

In [ ]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
from tools import *
import convert_examples_to_features

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
NUM_TRAIN_EPOCHS = 4
TASK_NAME = Data+Category
wb = torch.tensor(wb1) 

# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = f'/data/'
BERT_MODEL = 'bert-base-cased'
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_report/'

CACHE_DIR = 'cache/'

MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 12
EVAL_BATCH_SIZE = 12
LEARNING_RATE = 2e-5

RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [ ]:
output_mode = OUTPUT_MODE
cache_dir = CACHE_DIR

In [ ]:
if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(OUTPUT_DIR))
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

In [ ]:
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
print(label_list)

['0', '1']


In [ ]:
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [ ]:
label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

In [ ]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

Preparing to convert 320 examples..
Spawning 1 processes..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
with open(DATA_DIR + "train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

In [ ]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)
# model = BertForSequenceClassification.from_pretrained(CACHE_DIR + 'cased_base_bert_pytorch.tar.gz', cache_dir=CACHE_DIR, num_labels=num_labels)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir /tmp/tmpde74yjrf
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:pytorch_pretrained_bert.modeling:Weights of BertForSequenceClassification not initialized from pretrained model: ['clas

In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [ ]:
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=LEARNING_RATE,
                     warmup=WARMUP_PROPORTION,
                     t_total=num_train_optimization_steps)

In [ ]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [ ]:
logger.info("***** Running training *****")
logger.info("  Num examples = %d", train_examples_len)
logger.info("  Batch size = %d", TRAIN_BATCH_SIZE)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

all_input_ids = all_input_ids.to(device)
all_segment_ids = all_segment_ids.to(device)
all_input_mask = all_input_mask.to(device)
all_label_ids = all_label_ids.to(device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 320
INFO:root:  Batch size = 12
INFO:root:  Num steps = 104


In [ ]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [ ]:
model.train()
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        logits = model(input_ids, segment_ids, input_mask, labels=None)

        if OUTPUT_MODE == "classification":
            wb = wb.to(device)
            loss_fct = CrossEntropyLoss(weight= wb)
            loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


2.331972

In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(OUTPUT_DIR)

In [ ]:
import tarfile
import os
os.chdir(f'outputs/{TASK_NAME}/')
with tarfile.open(os.path.join('../../cache/', f"{TASK_NAME}.tar.gz"), "w:gz") as tar:
    for name in ["config.json", "pytorch_model.bin"]:
         tar.add(name)
tar.close()
os.chdir('../../')
os.getcwd()

In [ ]:
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)

In [ ]:
def get_eval_report(task_name, labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    #dfcm = pd.DataFrame(confusion_matrix(labels, preds),columns=[0,1],index=[0,1])
    #return(dfcm)
    return {
        "task": task_name,
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }

def compute_metrics(task_name, labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(task_name, labels, preds)

In [ ]:
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR + 'vocab.txt', do_lower_case=False)

In [ ]:
processor = BinaryClassificationProcessor()
eval_examples = processor.get_dev_examples(DATA_DIR)
label_list = processor.get_labels() # [0, 1] for binary classification
# for i in range(2,6):
#     label_list.append(str(i))
#label_list.append('2')
num_labels = len(label_list)
eval_examples_len = len(eval_examples)

In [ ]:
label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in eval_examples]

In [ ]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

In [ ]:
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)

In [ ]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

In [ ]:
model.to(device)

In [ ]:
import numpy as np
from sklearn.metrics import matthews_corrcoef, confusion_matrix
model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    # create eval loss and other metric required by the task
    wb = wb.to(device)
    loss_fct = CrossEntropyLoss(weight=wb)
    tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))

    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps
preds = preds[0]
preds = np.argmax(preds, axis=1)
result = compute_metrics(TASK_NAME, all_label_ids.numpy(), preds)
#print(result)
result['eval_loss'] = eval_loss


In [ ]:
Recall = result['tp']/(result['tp']+result['fn'])

Precision = result['tp']/(result['tp']+result['fp'])
f1 = 2/((1/Recall) + (1/Precision))

print(Precision, Recall, f1)

In [ ]:
print(result['tp'])
print(result['tn'])
print(result['fp'])
print(result['fn'])